In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

[BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/) documentation

In [2]:
with open("index.html", "r") as f:                # create a handle
    doc = BeautifulSoup(f, "html.parser")         # choose a parser

print(doc)    

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<title>Mi titulo</title>
</head>
<body bgcolor="FFFFFF">
<center><img align="bottom" src="miFoto.jpg"/></center>
<hr/>
<a href="http://wikipedia.org">Wikipedia</a>

		un link a otro lindo sitio
	<h1>Header h1</h1>
<h2>Header h2</h2>

		enviame un mail a <a =="" href="mailto:support@tutia.com">support mail</a>
<p> un parrafo</p>
<p> <b><i>Esta es una nueva oracion en italica y negrita sin un break de parrafo</i></b></p>
<hr/>
</body>
</html>


In [1]:
print(doc.prettify(formatter="html"))

In [4]:
tag = doc.title

In [5]:
print(tag)

<title>Mi titulo</title>


In [6]:
print(tag.string)
print(tag.text)

Mi titulo
Mi titulo


In [7]:
# Puedo cambiarle el valor al tag, pero como es de esperar no modifica el archivo original

In [8]:
tag.string = 'Hola'

In [9]:
tag.string

'Hola'

In [10]:
tag = doc.find('a')  # encuentra solo el primer tag de este tipo
tag

<a href="http://wikipedia.org">Wikipedia</a>

In [11]:
tags = doc.find_all('p') # me da un objeto que se comporta como un set
tags

[<p> un parrafo</p>,
 <p> <b><i>Esta es una nueva oracion en italica y negrita sin un break de parrafo</i></b></p>]

In [12]:
tags[1] # puedo seleccionar los diferentes elementos que encontré

<p> <b><i>Esta es una nueva oracion en italica y negrita sin un break de parrafo</i></b></p>

In [13]:
tags[1].find_all('b') # puedo buscar tags anidados

[<b><i>Esta es una nueva oracion en italica y negrita sin un break de parrafo</i></b>]

In [14]:
import requests

In [15]:
url = 'https://www.newegg.ca/gigabyte-geforce-rtx-3080-ti-gv-n308tgaming-oc-12gd/p/N82E16814932436?Description=3080&cm_re=3080-_-14-932-436-_-Product'

In [16]:
result = requests.get(url)

In [2]:
print(result.text)

In [18]:
doc = BeautifulSoup(result.text, 'html.parser')

In [3]:
print(doc.prettify())

In [20]:
prices = doc.find_all(text='$') # No solo se puede buscar tags, también texto

In [21]:
print(prices)

['$']


el precio tiene que verse algo así como __$xxxx__ o similar. Lo importante es que ya encontré el símbolo, si encuentro el tag padre de ese símbolo, puedo volver a bajar y encontrar todo el texto.

In [22]:
priceParent = prices[0].parent
priceParent

<li class="price-current"><span class="price-current-label"></span>$<strong>1,299</strong><sup>.99</sup></li>

In [23]:
precio = priceParent.find('strong').text
precio

'1,299'

## Vamos de nuevo con un archivo local

In [24]:
import re

In [25]:
with open('index2.html','r') as f:
    doc = BeautifulSoup(f,"html.parser")

In [26]:
tags = doc.find_all(['p','div','li']) # puedo buscar varios tags al mismo tiempo

In [27]:
#print(tags)

In [28]:
# puedo buscar tags que tengan ciertos valores
tags = doc.find_all(['option'], text="Undergraduate", value = 'undergraduate')

In [29]:
tags

[<option value="undergraduate">Undergraduate</option>]

In [30]:
print(tags[0].parent)

<select>
<option selected="" value="course-type">Course type*</option>
<option value="short-courses">Short courses</option>
<option value="featured-courses">Featured courses</option>
<option value="undergraduate">Undergraduate</option>
<option value="diploma">Diploma</option>
<option value="certificate">Certificate</option>
<option value="masters-degree">Masters degree</option>
<option value="postgraduate">Postgraduate</option>
</select>


In [31]:
# Puedo buscar por clases 

In [32]:
tags = doc.find_all(class_="btn-item")

In [33]:
tags

[<a class="btn-item" href="https://www.w3docs.com/learn-html.html">Learn HTML</a>,
 <a class="btn-item" href="https://www.w3docs.com/quiz/#">Select Quiz</a>]

#### usemos regex

In [34]:
tags = doc.find_all(text= re.compile("\$.*") )

In [35]:
tags

['\n        $2345\n      ', '\n        $123\n        ']

In [36]:
for tag in tags:
    print(tag.strip())

$2345
$123


#### Limitar el número de elementos que quiero encontrar

In [37]:
tags = doc.find_all(text=re.compile("\$.*"), limit = 1)

## Navegar a través del árbol HTML

In [38]:
url = 'https://coinmarketcap.com/'
reqs = requests.get(url)
doc = BeautifulSoup(reqs.text, 'html.parser')

In [39]:
tbody = doc.tbody

In [4]:
print(tbody)

**El tag tbody( cuerpo de una tabla) es padre de un montón de tags tr(table row).** 

**Para acceder a cada uno de ellos puedo hacerlo usando el método next_sibling()**

**Me puedo desplazar entre siblings aunque no sean del mismo tipo de tag**

In [5]:
# trs tiene todos los tags hijos de tbody
# los métodos siguientes son equivalentes .contents, .children
# creo que descendants va toooda la línea hacia abajo del arbol
trs = tbody.contents # sirve para ver el contenido
trs

In [42]:
# veamos el primer elemento
trs[0].a.text

'Bitcoin1BTC'

In [43]:
# veamos ahora el siguiente del primero
trs1 = trs[0].next_sibling.a.text

In [44]:
trs1

'Ethereum2ETH'

In [45]:
# Ahora posicionémonos en el segundo elemento y retrocedamos
trs[1].previous_sibling.a.text

'Bitcoin1BTC'

In [46]:
# Obtener todos los siblings siguientes
hijos = len(trs)
hijosSiguientes = list((trs[90].next_siblings))
print(f'Hay un total de {hijos} hijos. \nHay {len(hijosSiguientes)} después del elemento 91')

Hay un total de 100 hijos. 
Hay 9 después del elemento 91


#### Ahora volvamos al padre

In [47]:
trs[0].parent.name

'tbody'

In [48]:
prices = {}

In [6]:
for tr in trs:
    for td in tr.contents[2:4]:   # revisé como se ve, solo quiero el 2do y 3er td, nombre y precio
        print(f'{td}\n')

In [54]:
# vamos a hacerle un cambio al for interno porque ya vi que info tengo
# voy a declarar el el elemnto tr.contents[2] como nombre y el elemento tr.contents[3] como precio
# tr.contents[2:4] (la diferencia entre :4] y [3] es por la sintaxis de python)

In [60]:
for tr in trs[:10]:
    name, price = tr.contents[2:4]
    print(price.span.text)     # El nombre está dentro de un tag p =>> name.p.text
                               # El precio está dentro de un span q está dentro de un a

$17,428.44
$1,333.69
$1.00
$277.67
$1.00
$0.3628
$0.9999
$0.3137
$0.07654
$0.8582


In [63]:
# paso final en mi for:
for tr in trs[:10]:
    name, price = tr.contents[2:4]
    cleanedName = name.p.text
    cleanedPrice = price.span.text
    # Guardar las variables en un diccionario
    prices[cleanedName] = cleanedPrice     # Asignación key: value

In [64]:
prices

{'Bitcoin': '$17,428.44',
 'Ethereum': '$1,333.69',
 'Tether': '$1.00',
 'BNB': '$277.67',
 'USD Coin': '$1.00',
 'XRP': '$0.3628',
 'Binance USD': '$0.9999',
 'Cardano': '$0.3137',
 'Dogecoin': '$0.07654',
 'Polygon': '$0.8582'}